In [1]:
from dataloader.create_bias_dataset import CreateGenderStsb
import torch
import numpy as np
import torch
import pandas as pd
import math
import matplotlib.pyplot as plt
from transformers import T5ForConditionalGeneration,T5Tokenizer
import warnings; warnings.filterwarnings('ignore')

T5_PATH = 't5-large' # change the size of the model here. 
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
model = T5ForConditionalGeneration.from_pretrained(T5_PATH).to(DEVICE)

Some weights of the model checkpoint at t5-large were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The results derived from the experiments included in the 'run_bias_experiments.py' file, indicate that we should now
focus on these selected professions:

In [2]:
selected_careers = ['nurse', 'engineer', 'surgeon', 'scientist', 'receptionist', 'programmer', 'teacher', 'officer', 'homemaker']
#also added 'homemaker'

Create a df that contains identical sentences for each one of the aforementioned occupations and the words: 'she'
and 'he'

In [3]:
def create_dataset(occupation, data_dir=None, lang=None):
    dataset_creator = CreateGenderStsb(lang=lang, data_dir=data_dir, occupation=occupation, multilingual=False)
    women_df, men_df = dataset_creator.create_gendered_dataframes()
    df = pd.concat([
        women_df[['sentence1']],
        men_df[['sentence1', 'occupation']]
    ], axis=1)
    df.columns = ['she', 'he', f'{occupation}']
    return df


In [4]:
df1 = create_dataset('technician', data_dir='./data/stsbenchmark', lang='en')
for occupation in selected_careers:
    df = create_dataset(occupation, data_dir='./data/stsbenchmark', lang='en')
    df_all_occs = pd.concat([df1, df[[f'{occupation}']]], axis=1)
    df1 = df_all_occs
df = df1
df.to_csv('./selected_occs.csv', index=False)
df.reset_index(inplace=True)

len of test set: (1379, 7)
len of test set: (1379, 7)
len of test set: (1379, 7)
len of test set: (1379, 7)
len of test set: (1379, 7)
len of test set: (1379, 7)
len of test set: (1379, 7)
len of test set: (1379, 7)
len of test set: (1379, 7)
len of test set: (1379, 7)


In [5]:
any(df.duplicated()==True)

False

define functions to extract the embeddings from T5:

In [6]:
def get_sentence_tokens_and_embeds(sentence):
  """
  returns a tuple of size=len(tokens) populated by (token,embed)'s
  """
  input = str(sentence)
  labels = tokenizer('',return_tensors='pt').input_ids.to(DEVICE)
  ids = tokenizer(input, return_tensors='pt').input_ids.to(DEVICE)

  out = model(input_ids=ids,labels = labels, output_hidden_states=True)
  tokens = tokenizer.tokenize(input)
  #early_embeds = out.encoder_hidden_states[0]
  #early_embeds = early_embeds[:-1]
  embeds = out.encoder_last_hidden_state.squeeze(dim=0)
  embeds = embeds[:-1] # discard </s> embdedding. 
  tok_embeds = tuple(zip(tokens,embeds))
  return tok_embeds

def get_embed(tok_embeds, token):
  """
  given a single specific token, it returns its embedding
  exists only to be used in the get_occupation_embed method 
  """
  for i in range(len(tok_embeds)):
    tok = tok_embeds[i][0]
    embed = tok_embeds[i][1]
    if tok == token:
      # returns word's token alongside with corresponding embedding
      return tok, embed.cpu().detach().numpy()
    
def get_occupation_embed(tok_embeds, word):
  """
  given a word, it returns its embedding or 
  if the word has to be tokenized into more the one embeddings by the tokenizer, 
  it returns the sum/mean of those embeddings 
  (went with mean because this is better for measuring the angles later on,
  sum would change the result to a very big vector)
  """
  tokens = tokenizer.tokenize(word)
  if len(tokens)==1:
    return get_embed(tok_embeds, tokens[0])
  else:
    whole_word = ''
    embed_sum = np.zeros(model.config.hidden_size)
    for i in tok_embeds:
      if i[0] in tokens:
        whole_word += i[0]
        embed_sum += i[1].cpu().detach().numpy() # add rest of tokens if any 
    return (whole_word, embed_sum/len(tokens))

def dotproduct(v1, v2):
  return sum((a*b) for a, b in zip(v1, v2))

def length(v):
  return math.sqrt(dotproduct(v, v))
  
def angle(v1, v2):
  return (dotproduct(v1, v2) / (length(v1) * length(v2))) 


Populate the dataframe with the contextualized embeddings of 'he','she' and the occupations' embeddings.

In [ ]:
df['she_embed'] = df['she'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'She'))
df['he_embed'] = df['he'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'He'))
df['nurse_embed'] = df['nurse'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'nurse'))
df['engineer_embed'] = df['engineer'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'engineer'))
df['surgeon_embed'] = df['surgeon'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'surgeon'))
df['scientist_embed'] = df['scientist'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'scientist'))
df['receptionist_embed'] = df['receptionist'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'receptionist'))
df['programmer_embed'] = df['programmer'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'programmer'))
df['teacher_embed'] = df['teacher'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'teacher'))
df['homemaker_embed'] = df['homemaker'].apply(lambda x: get_occupation_embed(get_sentence_tokens_and_embeds(x), 'homemaker'))


Collect the embeddings as lists

In [ ]:
hes = [df.loc[i,'he_embed'][1] for i in range(len(df))]
shes = [df.loc[i,'she_embed'][1] for i in range(len(df))]
nurses = [df.loc[i,'nurse_embed'][1] for i in range(len(df))]
engineers = [df.loc[i,'engineer_embed'][1] for i in range(len(df))]
surgeons = [df.loc[i,'surgeon_embed'][1] for i in range(len(df))]
scientists = [df.loc[i,'scientist_embed'][1] for i in range(len(df))]
receptionists = [df.loc[i,'receptionist_embed'][1] for i in range(len(df))]
programmers = [df.loc[i,'programmer_embed'][1] for i in range(len(df))]
teachers = [df.loc[i,'teacher_embed'][1] for i in range(len(df))]
homemakers = [df.loc[i,'homemaker_embed'][1] for i in range(len(df))]

We explore the variation of the she-he distances and angles, to see whether a somewhat stable gender direction
exists in the embedding space.

In [ ]:
g_distances=[]
g_angles=[]
for i in range(len(hes)):
  d_i = shes[i]-hes[i]
  a_i = angle(shes[i],hes[i])
  g_distances.append(np.linalg.norm(d_i))
  g_angles.append(a_i)

mean_distance = np.mean(g_distances)
mean_angle = np.mean(g_angles)
print(f'she-he distance has a mean of {mean_distance} ± std {np.std(g_distances)}')
print(f'she-he angle has a mean of {mean_angle} ± std {np.std(g_angles)}')

We can also take a look at the variation of those angle-values on a polar graph.

In [ ]:
r = np.ones(len(g_angles))
theta = np.array(g_angles)
area = 40 * r**2
colors = theta

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(projection='polar')
ax.set_thetamin(0)
ax.set_thetamax(90)
ax.tick_params(axis='both', which='major', labelsize=25)
c = ax.scatter(theta, r, s=area, alpha=0.15, color='r') 

We will be using a mean g vector:

In [ ]:
g = np.zeros(model.config.hidden_size)
for i in range(len(hes)):
  g_i = hes[i] - shes[i]
  g += g_i
g = g/model.config.hidden_size

Define function to calculate b_i:

In [ ]:
def b_i(item_i, g):
  angles = []
  ang = 0
  for i in range(len(item_i)): 
    a = angle(item_i[i], g)
    ang += a
    mean_ang = (ang/len(item_i))
    angles.append(a)
  return angles, math.degrees(mean_ang)

Plot bi distributions for 'she' and 'he':

In [ ]:
plt.hist(b_i(hes,g)[0], bins=50, label='he', density=True, stacked=True, color='Blue')
plt.hist(b_i(shes,g)[0], bins=50, label='she', density=True, stacked=True, color='Grey')
plt.legend(loc='best')

Include the occupations:

In [ ]:
plt.rcParams["figure.figsize"] = (15,6)

plt.hist(b_i(hes,g)[0], bins=60, label='he', density=True, stacked=True, color='#377eb8')
plt.hist(b_i(shes,g)[0], bins=60, label='she', density=True, stacked=True, color='#ff7f00')
plt.hist(b_i(nurses,g)[0], bins=60, label='nurse', density=True, stacked=True, color='#4daf4a', alpha=0.7)
plt.hist(b_i(engineers,g)[0], bins=60, label='engineer', density=True, stacked=True, color='#f781bf',alpha=0.7)
plt.hist(b_i(surgeons,g)[0], bins=60, label='surgeon', density=True, stacked=True, color='#a65628',alpha=0.7)
plt.hist(b_i(scientists,g)[0], bins=60, label='scientist', density=True, stacked=True, color='#984ea3',alpha=0.7)
plt.hist(b_i(receptionists,g)[0], bins=60, label='receptionist', density=True, stacked=True, color='#999999',alpha=0.7)
plt.hist(b_i(programmers,g)[0], bins=60, label='programmer', density=True, stacked=True, color='#e41a1c',alpha=0.7)
plt.hist(b_i(teachers,g)[0], bins=60, label='teacher', density=True, stacked=True, color='#dede00',alpha=0.7)
plt.hist(b_i(homemakers,g)[0], bins=60, label='homemaker', density=True, stacked=True, color='black',alpha=0.7)

plt.legend(loc='best')
plt.title(T5_PATH)
plt.gca().axes.get_yaxis().set_visible(False)
plt.axvline(np.median(b_i(hes,g)[0]), linestyle='dashed', linewidth=2)
plt.axvline(np.median(b_i(shes,g)[0]), linestyle='dashed', linewidth=2)
plt.xlim(left=-0.5) 
plt.xlim(right=0.5) 

plt.show()